In [88]:
import numpy as np
import pandas as pd
import yfinance as yf

tickers = ['NKE', 'MSFT', 'DIS', 'COST']

start = '2012-1-1'
end = '2022-3-1'

df_close = yf.download(tickers, start, end, interval='1mo')['Close']

df_close.to_csv('./data/df_close.csv')

[*********************100%***********************]  4 of 4 completed


In [89]:
df_close.shape

(122, 4)

In [90]:
df_close.head()

,COST,DIS,MSFT,NKE
Date,,,,
2012-01-01 00:00:00-05:00,82.269997,38.900002,29.530001,25.997499
2012-02-01 00:00:00-05:00,86.059998,41.990002,31.740000,26.980000
2012-03-01 00:00:00-05:00,90.800003,43.779999,32.259998,27.110001
2012-04-01 00:00:00-04:00,88.180000,43.110001,32.020000,27.967501
2012-05-01 00:00:00-04:00,86.389999,45.709999,29.190001,27.045000


In [91]:
df_close.tail()

,COST,DIS,MSFT,NKE
Date,,,,
2021-10-01 00:00:00-04:00,491.540009,169.070007,331.619995,167.289993
2021-11-01 00:00:00-04:00,539.380005,144.899994,330.589996,169.240005
2021-12-01 00:00:00-05:00,567.700012,154.889999,336.320007,166.669998
2022-01-01 00:00:00-05:00,505.130005,142.970001,310.980011,148.070007
2022-02-01 00:00:00-05:00,519.250000,148.460007,298.790009,136.550003


In [92]:
df_close.isnull().sum()

COST    0
DIS     0
MSFT    0
NKE     0
dtype: int64

In [93]:
if df_close.isnull().values.any():
    df_close.fillna(method='ffill', inplace = True) #front fills
    df_close.fillna(method='bfill', inplace = True) #back fills

In [94]:
df_pct_change = df_close.pct_change().iloc[1:] #get rid of last row because cannot calc pct change for index +1
df_pct_change.head()

,COST,DIS,MSFT,NKE
Date,,,,
2012-02-01 00:00:00-05:00,0.046068,0.079434,0.074839,0.037792
2012-03-01 00:00:00-05:00,0.055078,0.042629,0.016383,0.004818
2012-04-01 00:00:00-04:00,-0.028855,-0.015304,-0.007439,0.031630
2012-05-01 00:00:00-04:00,-0.020299,0.060311,-0.088382,-0.032985
2012-06-01 00:00:00-04:00,0.099664,0.061037,0.047962,-0.188575


In [95]:
mean_return = df_pct_change.mean() * 12
mean_return

COST    0.198531
DIS     0.160302
MSFT    0.251882
NKE     0.190050
dtype: float64

In [96]:
var_risk = df_pct_change.var() * 12
var_risk

COST    0.029159
DIS     0.055223
MSFT    0.041740
NKE     0.049432
dtype: float64

In [97]:
cov_mat = df_pct_change.cov()*12
cov_mat

,COST,DIS,MSFT,NKE
COST,0.029159,0.011473,0.011932,0.012717
DIS,0.011473,0.055223,0.017132,0.020052
MSFT,0.011932,0.017132,0.041740,0.013990
NKE,0.012717,0.020052,0.013990,0.049432


In [98]:
corr_mat = df_pct_change.corr()
corr_mat

,COST,DIS,MSFT,NKE
COST,1.000000,0.285910,0.342005,0.334956
DIS,0.285910,1.000000,0.356845,0.383797
MSFT,0.342005,0.356845,1.000000,0.308000
NKE,0.334956,0.383797,0.308000,1.000000
